In [2]:
'''
This is code for extracting NN features of face image data 
and then fit a linear model to predict attractiveness of a face
Available dataset: TWIN, CHICAGO and MIT
Available NN feature: 'caffeNet','vgg16','vggFace' and 'faceSNN'

BY Linjie Li
Please run this code on guru2 server
'''
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import os

from sklearn.decomposition import PCA as sklearnPCA

# Load image dataset#
Dataset = 'mit' # 'twin', 'chicago' or 'mit', 'zhihu' or 'funnyFace'
if Dataset == 'twin':
    imPath = '../../processing/imageProcessing/paddedImages/'
    ext = '.png'
elif Dataset == 'chicago':
    imPath = '../../ChicagoFaceDataset/CFD Version 2.0/CFD 2.0 Images/'
    ext = 'N.jpg'
elif Dataset == 'mit':
    imPath = '../../MIT2kFaceDataset/2kfaces/'
    ext = '.jpg'
elif Dataset =='funnyFace':
    imPath = '../funnyFace/'
    ext = '.png'
else:
    imPath = '../../../zhihu/'
    ext = '.jpg'
# imList = []
# for dirpath, dirnames, filenames in os.walk(imPath):
#     for filename in [f for f in filenames if f.endswith(ext)]:
#         imList.append(os.path.join(dirpath, filename))
# imList.sort()
#print len(imList)
def readFile(fName):
    text_file = open(fName, "r")
    lines = text_file.read().split('\n')
    data = [None]*len(lines)
    for l in range(len(lines)):
        e = lines[l].split('.jpg ')[0]
        if(e):
            data[l] = e +'.jpg'
    text_file.close()
    return data
imList = readFile('../list/attr_train.txt')[:-1]
print len(imList)
testList = readFile('../list/attr_test.txt')[:-1]
print len(testList)
#print imPath
homePath = '/raid/linjieli/'
# Make sure that caffe is on the python path:
caffe_root = homePath+'caffe/'
pretrained_model_root = homePath+'caffe/'

# run this line one time only!
import sys
caffePython = pretrained_model_root + 'python'
if caffePython not in sys.path:
    sys.path.insert(0, caffePython)

import caffe
# Load mean
mu = np.load(caffe_root + 'python/caffe/imagenet/ilsvrc_2012_mean.npy')
mu = mu.mean(1).mean(1)  # average over pixels to obtain the mean (BGR) pixel values
print 'mean-subtracted values:', zip('BGR', mu)

# Load the trained net
MODEL = 'vgg16' #'caffeNet','vgg16','vggFace' or 'faceSNN'

saveFigPath = '../Result/'+Dataset+'/'+MODEL
if not os.path.exists(saveFigPath):
    os.makedirs(saveFigPath)
    
if MODEL == 'vgg16':
    MODEL_FILE = caffe_root +'models/VGG16/VGG_ILSVRC_16_layers_deploy.prototxt'
    PRETRAINED_FILE = caffe_root + 'models/VGG16/VGG_ILSVRC_16_layers.caffemodel'
elif MODEL == 'caffeNet':
    MODEL_FILE = caffe_root + 'models/bvlc_reference_caffenet/deploy.prototxt'
    PRETRAINED_FILE = caffe_root + 'models/bvlc_reference_caffenet/bvlc_reference_caffenet.caffemodel'
elif MODEL == 'vggFace':
    MODEL_FILE = caffe_root + 'models/VGGFACE/VGG_CNN_F_deploy.prototxt'
    PRETRAINED_FILE = caffe_root + 'models/VGGFACE/VGG_CNN_F.caffemodel'
    MEAN_FILE = caffe_root + 'models/VGGFACE/VGG_mean.binaryproto'
else:
    MODEL = 'faceSNN'
    MODEL_FILE = caffe_root +'models/sraonet/siamese_lecun_deploy.prototxt'
    PRETRAINED_FILE = caffe_root + 'models/sraonet/snapshots/sraonet_lecun_gd_sub2_iter_100000.caffemodel'
    
caffe.set_device(1)
caffe.set_mode_gpu()
if not os.path.isfile(PRETRAINED_FILE):
    print("No caffemodel!!!")
elif not os.path.isfile(MODEL_FILE):
    print("No MODEL !!!")
else:
    print "Defining the net!"
    net = caffe.Net(MODEL_FILE,
                PRETRAINED_FILE,
                caffe.TEST)

# input preprocessing: 'data' is the name of the input blob == net.inputs[0]
transformer = caffe.io.Transformer({'data': net.blobs['data'].data.shape})
if MODEL != 'faceSNN':
    # subtract the dataset-mean value in each channel
    transformer.set_mean('data', mu)
for layer_name, param in net.params.iteritems():
    print layer_name + '\t' + str(param[0].data.shape), str(param[1].data.shape)
transformer.set_transpose('data', (2,0,1))
# the reference model operates on images in [0,255] range instead of [0,1]
transformer.set_raw_scale('data', 255) 
# the reference model has channels in BGR order instead of RGB
transformer.set_channel_swap('data', (2,1,0))

plt.rcParams['figure.figsize'] = (10, 10)
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

if MODEL == 'vgg16' or MODEL == 'vggFace':
    imgeReshape = [224,224]
    featureLayer = 'conv5_2' 
elif MODEL == 'caffeNet':
    imgeReshape = [227,227]
    featureLayer = 'fc6'
else:
    imgeReshape = [56,46]
    featureLayer = 'fc6'
if 'fc' in featureLayer:
    featureNum = net.params[featureLayer][1].data.shape[0]
else:
    featureNum = net.blobs[featureLayer].data.flatten().shape[0]/net.blobs[featureLayer].data.shape[0]
    
if Dataset == 'twin':
    features = np.zeros([4,len(imList)/4,featureNum])
    perImNum = len(imList)/4
    img_type_num = {}
    img_type_index = {}
    img_type_list = {}
    type_index = 0
else:
    features = np.zeros([len(imList),featureNum])


def getFeature(imList,features):
    totalNum = 0
    for img in imList:
        if(img):
            imgName = os.path.basename(img)
            if imgName.endswith(('.jpg','.png'))and imgName:
                input_image = caffe.io.load_image(img)
                net.blobs['data'].reshape(1,3,imgeReshape[0],imgeReshape[1])
                net.blobs['data'].data[...] = transformer.preprocess('data', input_image)
                out = net.forward()
                feat = net.blobs[featureLayer].data
                if Dataset =='twin':
                    img_type = int(imgName[7:-4])/perImNum
                    img_index = int(imgName[7:-4])%perImNum
                    #print 'img_type:',img_type
                    if img_type in img_type_num.keys():
                        img_type_num[img_type] = img_type_num[img_type] + 1
                        img_type_list[img_type][img_index] = img
                    else:
                        img_type_num[img_type] = 0
                        img_type_list[img_type] = [None]*perImNum
                        img_type_index[img_type] = type_index
                        type_index +=1
                    #print 'img_type_index:',img_type_index[img_type]
                    features[img_type_index[img_type],img_type_num[img_type]] = feat.flatten()
                else:
                    # need to be further revised!
                    features[totalNum] = feat.flatten()
                    #print features[totalNum]
                totalNum +=1
            else:
                print img
    print totalNum
    return features
trainFeatures = getFeature(imList,features)
testFeatures = np.zeros([len(testList),featureNum])
testFeatures = getFeature(testList,testFeatures)

#print img_type_list

# if Dataset == 'twin':
#     featureMat = np.zeros((totalNum,featureNum))
#     k = 0
#     for i in range(features.shape[0]):
#         for j in range(features[i].shape[0]):
#             if sum(features[i,j,:])!=0:
#                 featureMat[k,:] = features[i,j,:]
#                 k +=1
# else:

#print featureMat

1777
445
mean-subtracted values: [('B', 104.0069879317889), ('G', 116.66876761696767), ('R', 122.6789143406786)]
Defining the net!
conv1_1	(64, 3, 3, 3) (64,)
conv1_2	(64, 64, 3, 3) (64,)
conv2_1	(128, 64, 3, 3) (128,)
conv2_2	(128, 128, 3, 3) (128,)
conv3_1	(256, 128, 3, 3) (256,)
conv3_2	(256, 256, 3, 3) (256,)
conv3_3	(256, 256, 3, 3) (256,)
conv4_1	(512, 256, 3, 3) (512,)
conv4_2	(512, 512, 3, 3) (512,)
conv4_3	(512, 512, 3, 3) (512,)
conv5_1	(512, 512, 3, 3) (512,)
conv5_2	(512, 512, 3, 3) (512,)
conv5_3	(512, 512, 3, 3) (512,)
fc6	(4096, 25088) (4096,)
fc7	(4096, 4096) (4096,)
fc8	(1000, 4096) (1000,)
1777
445


In [6]:
trainfeatureMat = trainFeatures
ratio = 0.1
numVali = int(trainfeatureMat.shape[0]*ratio)
print numVali
valifeature = trainfeatureMat[:numVali,:]
trainfeatureMat = trainfeatureMat[numVali:,:]
print trainfeatureMat.shape[0]

177
1600


In [9]:
if MODEL != 'faceSNN':
    explained_variance = 0.95
else:
    explained_variance = featureNum

sklearn_pca = sklearnPCA(n_components=explained_variance, whiten  = True)
trainfeature_transf = sklearn_pca.fit_transform(trainfeatureMat)
print 'The number of PCs needed to retain %.3f variance is %d.' \
      % (explained_variance, trainfeature_transf.shape[1])

print sklearn_pca.components_.shape
print trainfeature_transf.shape
testfeature_tansf = sklearn_pca.transform(testFeatures)
print testfeature_tansf.shape
valifeature_transf = sklearn_pca.transform(valifeature)
#component_std = np.std(sklearn_pca.components_,axis = 0,dtype=np.float32)
# import scipy
# u,s,v = scipy.sparse.linalg.svds(featureMat, k =sklearn_pca.components_.shape[0], which = 'LM')
np.savetxt(homePath+'/features/'+MODEL+'_'+featureLayer+'_'+Dataset+\
           '_trainFeatures.csv', trainfeature_transf, delimiter=',')
np.savetxt(homePath+'/features/'+MODEL+'_'+featureLayer+'_'+Dataset+\
           '_testFeatures.csv', testfeature_tansf, delimiter=',')
np.savetxt(homePath+'/features/'+MODEL+'_'+featureLayer+'_'+Dataset+\
           '_valiFeatures.csv', valifeature_transf, delimiter=',')

n_samples = trainfeatureMat.shape[0]
# component_std = s*n_samples
#print component_std
# component_std.shape = (component_std.shape[0],1)

whiten_component = sklearn_pca.components_
#print whiten_component
newMat = trainfeatureMat.dot(whiten_component.T)
bias = np.mean(newMat,axis = 0)
# print bias
newNewMat = newMat - bias

np.savetxt(homePath+'/features/'+MODEL+'_'+featureLayer+'_'+Dataset+\
           '_weights.csv', whiten_component, delimiter=',')
np.savetxt(homePath+'/features/'+MODEL+'_'+featureLayer+'_'+Dataset+\
           '_biases.csv', bias, delimiter=',')


The number of PCs needed to retain 0.950 variance is 942.
(942, 100352)
(1600, 942)
(445, 942)
